# Playground
Assuming that semio was installed over the whl. In order to do so run,
`poetry build` in `src/python/core`


In [31]:
from semio.geometry import Point
from semio.model import Pose,Sobject, Platform, Parameter, Value, Representation,Plan
from semio.utils import MessageToJson, hashObject

parameter = Parameter(name="Length",value=Value(number=23.456))
json = MessageToJson(parameter)
json

p1 = Plan(uri="Test",parameters=[parameter])
p2 = Plan(uri="Test")
set((hashObject(p1),hashObject(p2)))


s1 = Sobject(plan=p1)
s2 = Sobject(plan=p2)
type(s1.ListFields()[0][1:][0])

model.v1.model_pb2.Plan

In [ ]:
s3 = Sobject(**)
s3

In [68]:
from multipledispatch import dispatch

from semio.geometry import Point,Quaternion
from semio.model import Pose,Sobject

from mathutils import Vector as BlenderVector, Quaternion as BlenderQuaternion,Euler as BlenderEuler ,  Matrix as BlenderMatrix
from numpy import shape,array,matmul,dot, allclose

class BlenderMathConverter:

    # Semio to Blender
    @dispatch(Quaternion)
    @staticmethod
    def convert(view:Quaternion):
        return BlenderQuaternion((view.w,view.x,view.y,view.z))

    @dispatch(Point)
    @staticmethod
    def convert(point:Point):
        return BlenderVector((point.x,point.y,point.z))

    @dispatch(Pose)
    @staticmethod
    def convert(pose:Pose):
        origin = convert(pose.point_of_view)
        quaternion = convert(pose.view)
        matrix = quaternion.to_matrix().to_4x4()
        matrix[0][3]=origin.x
        matrix[1][3]=origin.y
        matrix[2][3]=origin.z
        return matrix

    # Blender to Semio

    @dispatch(BlenderVector)
    @staticmethod
    def convert(vector:Point):
        return Point(x=vector.x,y=vector.y,z=vector.z)

def applyTransforms(vector:BlenderVector,transform:BlenderMatrix):
        """Apply transformation to a point of view."""
        return Vector(matmul(array(transform),array(vector.to_4d())))

def getLocalPointOfView(pose:Pose ,worldPointOfView:Point, considerPointOfView = True, considerView = True)->Point:
    """
    Get another point of view from a world perspective in a local perspective.
    Parameters:
    worldPointOfViewLike: Point of view from world view.
    """
    transformedPointOfView = BlenderMathConverter.convert(worldPointOfView)
    if considerPointOfView:
        transformedPointOfView = applyTransforms(transformedPointOfView,BlenderMatrix.Translation(-BlenderMathConverter.convert(pose.point_of_view)))
    if considerView:
        transformedPointOfView = applyTransforms(transformedPointOfView,BlenderMathConverter.convert(pose.view).to_matrix().to_4x4())
    return convert(transformedPointOfView)

def getWorldPointOfView(pose:Pose, localPointOfView:Point, considerPointOfView = True, considerView = True)->Point:
    """
    Get another point of view from a local perspective in a world perspective.
    Parameters:
    localPointOfView: Point of view from the local view.
    """
    transformedPointOfView = convert(localPointOfView)
    if considerView:
        transformedPointOfView = applyTransforms(transformedPointOfView,BlenderMathConverter.convert(pose.view).inverted().to_matrix().to_4x4())
    if considerPointOfView:
        transformedPointOfView = applyTransforms(transformedPointOfView,BlenderMatrix.Translation(BlenderMathConverter.convert(pose.point_of_view)))
    return BlenderMathConverter.convert(transformedPointOfView)

In [69]:
o = Point(x=600,y=180,z=300)
q = Quaternion(w=0.558,x=-0.681,y=0.247,z=-0.404)
p = Pose(point_of_view=o,view=q)

p1=Point()
l = getLocalPointOfView(p,Point())
w = getWorldPointOfView(p,p1)
p1Test = getLocalPointOfView(p,w)
p1Test2 = getWorldPointOfView(p,l)
p1Test2

x: 0.2132568359375
y: -0.055145263671875
z: -0.3931884765625